In [1]:
from tensorflow.keras.models import load_model

In [3]:
path=r"model weights"

In [4]:
mod=load_model(path)

In [5]:
import cv2

In [6]:
proto=r'face_detector\deploy.prototxt'
caf=r'face_detector\res10_300x300_ssd_iter_140000.caffemodel'
facenet=cv2.dnn.readNet(proto, caf)

In [7]:
import numpy as np

In [8]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
def predictor(frame, facenet, mod):
    (h,w)=frame.shape[:2]
    blob=cv2.dnn.blobFromImage(frame, 1, (224,224), (104.0, 177.0, 123.0))
    facenet.setInput(blob)
    
    t=facenet.forward()
    
    faces=[]
    locs=[]
    preds=[]
    for i in range(t.shape[2]):
        
        conf=t[0,0,i,2]
        if conf>0.5:
            box=t[0,0,i,3:7]*np.array([w,h,w,h])
            (startx, starty, endx, endy)=box.astype("int")
            face=frame[startx:endx, starty:endy]
            face=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            face=cv2.resize(frame, (224,224))
            face=img_to_array(face)
            face=preprocess_input(face)
            faces.append(face)
            locs.append(box.astype("int"))
    if len(faces)>0:
        preds=mod.predict(np.array(faces, dtype="float32"))
    return (preds, locs)     

            



    

In [9]:
from imutils.video import VideoStream
vs= cv2.VideoCapture(0)

while True:
    a,frame=vs.read()
    (preds, locs)=predictor(frame, facenet, mod)
    for i ,j in zip(preds, locs):
        mask=i.astype("float32")
        notmask=1-mask
        print(mask)
        (x, y, h, w)=j
        label="Mask" if mask<0.5 else "No Mask"
        color=(0,0,255) if label=="No Mask" else (0,255,0)
        textshow=label+str(max(mask, notmask)*100)
        cv2.rectangle(frame, (x,y), (h,w), color, 3)
        cv2.putText(frame, textshow, (x, y-10), cv2.FONT_HERSHEY_COMPLEX, 0.45, color, 2)
    cv2.imshow("Frame", frame)
    key=cv2.waitKey(1) & 0xFF
    if key==ord('a'):
        break

cv2.destroyAllWindows()
# vs.stream.release()
vs.release()
# vs.stop()

[9.653617e-25]
[0.]
[1.3587136e-36]
[0.]
[2.3591095e-37]
[1.005597e-29]
[1.0353006e-28]
[5.4682273e-17]
[1.6525293e-13]
[4.8555537e-09]
[1.7624767e-13]
[1.]
[1.]
[0.9999994]
[0.99987763]
[0.9999974]
[0.999998]
[0.99983394]
[1.]
[1.]
[0.2652159]
[0.9999739]
[0.00448278]
[0.9919426]
[1.]
[1.]
[1.]
[0.9986054]
[1.0801202e-22]
[3.9953872e-13]
[1.5769807e-13]
